# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [15]:
import numpy as np
import pandas as pd
import xlsxwriter
import requests
from scipy import stats
import math

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [16]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [17]:
symbol = 'aapl'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

{'symbol': 'AAPL', 'companyName': 'Apple Inc', 'primaryExchange': 'EOARSDLLNM/ BALAS KATQS)CGTN(G EE', 'calculationPrice': 'close', 'open': None, 'openTime': None, 'openSource': 'afcifilo', 'close': None, 'closeTime': None, 'closeSource': 'aiofcfli', 'high': None, 'highTime': None, 'highSource': None, 'low': None, 'lowTime': None, 'lowSource': None, 'latestPrice': 128.65, 'latestSource': 'Close', 'latestTime': 'April 6, 2021', 'latestUpdate': 1624193805717, 'latestVolume': None, 'iexRealtimePrice': 127.52, 'iexRealtimeSize': 10, 'iexLastUpdated': 1670164514254, 'delayedPrice': None, 'delayedPriceTime': None, 'oddLotDelayedPrice': None, 'oddLotDelayedPriceTime': None, 'extendedPrice': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPriceTime': None, 'previousClose': 128.9, 'previousVolume': 93071988, 'change': 0.32, 'changePercent': 0.00247, 'volume': None, 'iexMarketPercent': 110.36919532686943, 'iexVolume': 1316143, 'avgTotalVolume': 105257537, 'iexBidPrice': 0, 

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [18]:
price = data['latestPrice']
pe_ratio = data['peRatio']
pe_ratio

35.5

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [19]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [20]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['quote']['latestPrice'],
                    data[symbol]['quote']['peRatio'],
                    'N/A'
                ],
                index=my_columns
            ),
            ignore_index=True
        )
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,133.80,51.7,N/A
1,AAL,24.77,-1.35,N/A
2,AAP,188.68,26.12,N/A
3,AAPL,127.38,34.7,N/A
4,ABBV,110.13,38.85,N/A
...,...,...,...,...
500,YUM,114.90,39,N/A
501,ZBH,167.90,-250.72,N/A
502,ZBRA,510.97,53.62,N/A
503,ZION,57.42,9.12,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [21]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace=True)
final_dataframe50 = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe50 = final_dataframe50[:50]
final_dataframe50.reset_index(inplace=True)
final_dataframe50.drop('index', axis=1, inplace=True)
final_dataframe50

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,BIO,615.65,4.8,N/A
1,ALL,123.22,6.82,N/A
2,UNM,28.45,7.46,N/A
3,AFL,51.82,7.83,N/A
4,EBAY,62.89,8.19,N/A
5,COO,407.04,8.69,N/A
6,KIM,20.40,8.98,N/A
7,ZION,57.42,9.12,N/A
8,CE,159.90,9.43,N/A
9,CTL,12.00,9.89,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [22]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [23]:
portfolio_input()

You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [29]:
position_size = float(portfolio_size) / len(final_dataframe50.index)
for row in final_dataframe50.index:
    final_dataframe50.loc[row, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe50.loc[row, 'Price'])

final_dataframe50

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,BIO,615.65,4.8,32
1,ALL,123.22,6.82,162
2,UNM,28.45,7.46,702
3,AFL,51.82,7.83,385
4,EBAY,62.89,8.19,318
5,COO,407.04,8.69,49
6,KIM,20.40,8.98,980
7,ZION,57.42,9.12,348
8,CE,159.90,9.43,125
9,CTL,12.00,9.89,1666


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [39]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()
print(data['AAPL']['advanced-stats'])
print(data['AAPL']['advanced-stats']['enterpriseValue'])

# Price-to-earnings ratio
pe_ratio = data[symbol]['quote']['peRatio']

# Price-to-book ratio
pb_ratio = data['AAPL']['advanced-stats']['priceToBook']

# Price-to-sales ratio
ps_ratio = data['AAPL']['advanced-stats']['priceToSales']

# Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
enterprise_value = data['AAPL']['advanced-stats']['enterpriseValue']
ebitda = data['AAPL']['advanced-stats']['EBITDA']
ev_to_ebitda =  enterprise_value / ebitda

# Enterprise Value divided by Gross Profit (EV/GP)
gross_profit = data['AAPL']['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value / gross_profit

{'beta': 1.2690042908984522, 'totalCash': 80377646739, 'currentDebt': 135248795706, 'revenue': 307079743952, 'grossProfit': 119179158779, 'totalRevenue': 300729080041, 'EBITDA': 87279433277, 'revenuePerShare': 18.22, 'revenuePerEmployee': 2033089.2, 'debtToEquity': 5.39666181734507, 'profitMargin': 0.22526746348009843, 'enterpriseValue': 2283414432933, 'enterpriseValueToRevenue': 7.68, 'priceToSales': 7.6, 'priceToBook': 32.86, 'forwardPERatio': 29.09542166214406, 'pegRatio': 2.0283211913186143, 'peHigh': 41.69365338617663, 'peLow': 17.019589133697668, 'week52highDate': '2021-01-24', 'week52lowDate': '2020-03-31', 'putCallRatio': 0.3717810633681994, 'companyName': 'Apple Inc', 'marketcap': 2250621161537, 'week52high': 149.13, 'week52low': 67.66, 'week52highSplitAdjustOnly': 148.58, 'week52highDateSplitAdjustOnly': '2021-01-23', 'week52lowSplitAdjustOnly': 66.92, 'week52lowDateSplitAdjustOnly': '2020-03-29', 'week52change': 0.9699097547488701, 'sharesOutstanding': 17410935113, 'float': 

Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [47]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'Price-to-Earning Ratio',
    'PE Percentile',
    'Price-To-Book Ratio',
    'PB Percentile',
    'Price-To-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns=rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']

        try:
            ev_to_ebitda = enterprise_value / ebitda
        except TypeError:
            ev_to_ebitda = np.NaN

        try:
            ev_to_gross_profit = enterprise_value / gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN

        rv_dataframe = rv_dataframe.append(
            pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gross_profit,
                'N/A',
                'N/A'
            ],
            index=rv_columns),
            ignore_index=True
        )

In [48]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-To-Book Ratio,PB Percentile,Price-To-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,132.60,N/A,52.39,N/A,8.47,N/A,7.3,N/A,32.934139,N/A,14.217959,N/A,N/A
1,AAL,24.03,N/A,-1.34,N/A,-2.23,N/A,0.9028,N/A,-4.531984,N/A,2.363462,N/A,N/A
2,AAP,181.10,N/A,26.44,N/A,3.4,N/A,1.19,N/A,11.865412,N/A,2.595032,N/A,N/A
3,AAPL,133.39,N/A,35.7,N/A,34.63,N/A,7.5,N/A,25.724384,N/A,19.461464,N/A,N/A
4,ABBV,109.00,N/A,40.27,N/A,14.86,N/A,4.11,N/A,14.355243,N/A,8.923425,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,117.84,N/A,39,N/A,-4.4,N/A,6.34,N/A,24.735557,N/A,10.810684,N/A,N/A
501,ZBH,166.59,N/A,-249.66,N/A,2.94,N/A,5.02,N/A,30.988767,N/A,8.504125,N/A,N/A
502,ZBRA,519.20,N/A,53.85,N/A,13.05,N/A,6.32,N/A,37.166391,N/A,13.987971,N/A,N/A
503,ZION,57.00,N/A,8.88,N/A,1.29,N/A,2.67,N/A,6.714781,N/A,2.955355,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [54]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-To-Book Ratio,PB Percentile,Price-To-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
40,AON,244.00,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
42,APA,18.01,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,273.85,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11.00,N/A,9.83,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
136,DISCK,36.78,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
165,ETFC,51.28,N/A,14.83,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
168,EVRG,62.90,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
190,FOX,36.18,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
192,FRC,174.08,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
204,GOOG,2361.72,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [61]:
for column in ['Price-to-Earning Ratio', 'Price-To-Book Ratio', 'Price-To-Sales Ratio', 'EV/EBITDA', 'EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace=True)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-To-Book Ratio,PB Percentile,Price-To-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,132.60,N/A,52.39,N/A,8.47,N/A,7.3000,N/A,32.934139,N/A,14.217959,N/A,N/A
1,AAL,24.03,N/A,-1.34,N/A,-2.23,N/A,0.9028,N/A,-4.531984,N/A,2.363462,N/A,N/A
2,AAP,181.10,N/A,26.44,N/A,3.40,N/A,1.1900,N/A,11.865412,N/A,2.595032,N/A,N/A
3,AAPL,133.39,N/A,35.70,N/A,34.63,N/A,7.5000,N/A,25.724384,N/A,19.461464,N/A,N/A
4,ABBV,109.00,N/A,40.27,N/A,14.86,N/A,4.1100,N/A,14.355243,N/A,8.923425,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,117.84,N/A,39.00,N/A,-4.40,N/A,6.3400,N/A,24.735557,N/A,10.810684,N/A,N/A
501,ZBH,166.59,N/A,-249.66,N/A,2.94,N/A,5.0200,N/A,30.988767,N/A,8.504125,N/A,N/A
502,ZBRA,519.20,N/A,53.85,N/A,13.05,N/A,6.3200,N/A,37.166391,N/A,13.987971,N/A,N/A
503,ZION,57.00,N/A,8.88,N/A,1.29,N/A,2.6700,N/A,6.714781,N/A,2.955355,N/A,N/A


Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [62]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-To-Book Ratio,PB Percentile,Price-To-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [68]:
from scipy.stats import percentileofscore as score

metrics = {
    'Price-to-Earning Ratio': 'PE Percentile',
    'Price-To-Book Ratio': 'PB Percentile',
    'Price-To-Sales Ratio': 'PS Percentile',
    'EV/EBITDA': 'EV/EBITDA Percentile',
    'EV/GP': 'EV/GP Percentile'
}

for metric in metrics.keys():
    for row in rv_dataframe.index:
        rv_dataframe.loc[row, metrics[metric]] = score(rv_dataframe[metric], rv_dataframe.loc[row, metric])

rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-To-Book Ratio,PB Percentile,Price-To-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,132.60,N/A,52.39,83.960396,8.47,73.762376,7.3000,79.009901,32.934139,87.326733,14.217959,79.80198,N/A
1,AAL,24.03,N/A,-1.34,15.247525,-2.23,4.554455,0.9028,7.326733,-4.531984,3.960396,2.363462,8.514851,N/A
2,AAP,181.10,N/A,26.44,48.118812,3.40,42.772277,1.1900,13.168317,11.865412,24.356436,2.595032,10.49505,N/A
3,AAPL,133.39,N/A,35.70,68.514851,34.63,95.247525,7.5000,79.60396,25.724384,77.425743,19.461464,91.287129,N/A
4,ABBV,109.00,N/A,40.27,74.653465,14.86,87.425743,4.1100,55.247525,14.355243,37.425743,8.923425,52.673267,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,117.84,N/A,39.00,72.673267,-4.40,4.356436,6.3400,74.851485,24.735557,75.445545,10.810684,66.732673,N/A
501,ZBH,166.59,N/A,-249.66,0.990099,2.94,36.732673,5.0200,63.168317,30.988767,85.742574,8.504125,49.108911,N/A
502,ZBRA,519.20,N/A,53.85,84.356436,13.05,85.544554,6.3200,74.554455,37.166391,88.316832,13.987971,78.217822,N/A
503,ZION,57.00,N/A,8.88,17.425743,1.29,11.089109,2.6700,38.019802,6.714781,6.138614,2.955355,12.079208,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [73]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-To-Book Ratio,PB Percentile,Price-To-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,132.60,N/A,52.39,83.960396,8.47,73.762376,7.3000,79.009901,32.934139,87.326733,14.217959,79.80198,80.772277
1,AAL,24.03,N/A,-1.34,15.247525,-2.23,4.554455,0.9028,7.326733,-4.531984,3.960396,2.363462,8.514851,7.920792
2,AAP,181.10,N/A,26.44,48.118812,3.40,42.772277,1.1900,13.168317,11.865412,24.356436,2.595032,10.49505,27.782178
3,AAPL,133.39,N/A,35.70,68.514851,34.63,95.247525,7.5000,79.60396,25.724384,77.425743,19.461464,91.287129,82.415842
4,ABBV,109.00,N/A,40.27,74.653465,14.86,87.425743,4.1100,55.247525,14.355243,37.425743,8.923425,52.673267,61.485149
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,117.84,N/A,39.00,72.673267,-4.40,4.356436,6.3400,74.851485,24.735557,75.445545,10.810684,66.732673,58.811881
501,ZBH,166.59,N/A,-249.66,0.990099,2.94,36.732673,5.0200,63.168317,30.988767,85.742574,8.504125,49.108911,47.148515
502,ZBRA,519.20,N/A,53.85,84.356436,13.05,85.544554,6.3200,74.554455,37.166391,88.316832,13.987971,78.217822,82.19802
503,ZION,57.00,N/A,8.88,17.425743,1.29,11.089109,2.6700,38.019802,6.714781,6.138614,2.955355,12.079208,16.950495


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [79]:
rv_dataframe.sort_values('RV Score', ascending=True, inplace=True)
rv_dataframe50 = rv_dataframe[:50]
rv_dataframe50.reset_index(drop=True, inplace=True)
rv_dataframe50

,Ticker,Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-To-Book Ratio,PB Percentile,Price-To-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,28.51,N/A,7.00,16.237624,0.5127,4.752475,0.4244,1.980198,3.416881,4.752475,0.404628,0.39604,5.623762
1,MCK,196.84,N/A,-7.86,10.891089,-66.0400,0.39604,0.1291,0.594059,7.282359,6.732673,2.838550,11.089109,5.940594
2,FTI,8.04,N/A,-1.08,15.643564,0.8457,6.138614,0.2676,0.990099,2.304105,4.356436,1.454354,4.554455,6.336634
3,AIG,47.16,N/A,-6.80,11.980198,0.6085,5.346535,0.9402,8.118812,7.330414,6.930693,0.906402,1.980198,6.871287
4,MET,62.21,N/A,11.32,19.80198,0.7456,5.544554,0.8365,6.534653,5.464028,5.148515,0.807354,1.584158,7.722772
5,AAL,24.03,N/A,-1.34,15.247525,-2.2300,4.554455,0.9028,7.326733,-4.531984,3.960396,2.363462,8.514851,7.920792
6,ALL,117.97,N/A,7.09,16.435644,1.3000,11.584158,0.8265,6.336634,2.587438,4.554455,0.806668,1.386139,8.059406
7,F,13.00,N/A,-39.94,5.148515,1.6000,17.425743,0.3949,1.782178,6.169405,5.742574,2.883367,11.683168,8.356436
8,ABC,120.93,N/A,-7.39,11.089109,-35.6000,1.188119,0.1225,0.39604,8.888389,12.079208,4.207838,20.594059,9.069307
9,DXC,32.00,N/A,-2.90,13.663366,1.4400,15.049505,0.4438,2.178218,6.067257,5.544554,2.643031,10.693069,9.425743


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [83]:
portfolio_input()

In [94]:
position_size = float(portfolio_size) / len(rv_dataframe50.index)

for row in rv_dataframe50.index:
    rv_dataframe50.loc[row, 'Number of Shares to Buy'] = math.floor(position_size / rv_dataframe50.loc[row, 'Price'])

rv_dataframe50

d:\tutorial\algorithmic-trading-python\.virtualenv\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,Price-to-Earning Ratio,PE Percentile,Price-To-Book Ratio,PB Percentile,Price-To-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,28.51,7015,7.00,16.237624,0.5127,4.752475,0.4244,1.980198,3.416881,4.752475,0.404628,0.39604,5.623762
1,MCK,196.84,1016,-7.86,10.891089,-66.0400,0.39604,0.1291,0.594059,7.282359,6.732673,2.838550,11.089109,5.940594
2,FTI,8.04,24875,-1.08,15.643564,0.8457,6.138614,0.2676,0.990099,2.304105,4.356436,1.454354,4.554455,6.336634
3,AIG,47.16,4240,-6.80,11.980198,0.6085,5.346535,0.9402,8.118812,7.330414,6.930693,0.906402,1.980198,6.871287
4,MET,62.21,3214,11.32,19.80198,0.7456,5.544554,0.8365,6.534653,5.464028,5.148515,0.807354,1.584158,7.722772
5,AAL,24.03,8322,-1.34,15.247525,-2.2300,4.554455,0.9028,7.326733,-4.531984,3.960396,2.363462,8.514851,7.920792
6,ALL,117.97,1695,7.09,16.435644,1.3000,11.584158,0.8265,6.336634,2.587438,4.554455,0.806668,1.386139,8.059406
7,F,13.00,15384,-39.94,5.148515,1.6000,17.425743,0.3949,1.782178,6.169405,5.742574,2.883367,11.683168,8.356436
8,ABC,120.93,1653,-7.39,11.089109,-35.6000,1.188119,0.1225,0.39604,8.888389,12.079208,4.207838,20.594059,9.069307
9,DXC,32.00,6250,-2.90,13.663366,1.4400,15.049505,0.4438,2.178218,6.067257,5.544554,2.643031,10.693069,9.425743


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [22]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index=False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

## Saving Our Excel Output
As before, saving our Excel output is very easy: